In [28]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

In [29]:
# load the ANN trained model
model=load_model("model.h5")

# load the gender pkl file 
with open('label_encoder_Gender.pkl', 'rb') as file:
    label_encoder_Gender = pickle.load(file)

# load the one hot encoder pkl file
with open('onehot_encoder_geography.pkl', 'rb') as file:
    onehot_encoder_geography = pickle.load(file)

# load the scaler pkl file
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [30]:
# Example input data for prediction
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [31]:
# converting input data to dataframe
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [32]:
# One Hot encode 'Geography'
geo_encoded = onehot_encoder_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, 
             columns=onehot_encoder_geography.get_feature_names_out(['Geography']))

d:\AI ML (GEN AI)\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [33]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [34]:
# Encode categorical variables
input_df['Gender'] = label_encoder_Gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [35]:
# concatenation with one hot encoded geography columns
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [36]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [37]:
# scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [38]:
# predict churn 
model_prediction = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step


In [39]:
model_prediction

array([[0.03556092]], dtype=float32)

In [41]:
# get the predition probabality
prediction_probability = model_prediction[0][0]

In [42]:
prediction_probability

np.float32(0.035560925)

In [43]:
# get the prediction probabality
if prediction_probability >= 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
